In [1]:
import spacy
from spacy import displacy
import requests

nlp = spacy.load("en_core_web_sm")

# load wikidata API
url = 'https://www.wikidata.org/w/api.php'
# a header to get access to more requests
headers = {'User-Agent': 'NicBot/0.0 (N.c.brand@student.rug.nl)'}

# json parameters for entities
entParams = {'action':'wbsearchentities', 
          'language':'en',
          'limit':'2',
          'format':'json'}

# json parameters for properties
propParams = {'action':'wbsearchentities', 
          'type':'property',
          'language':'en',
          'limit':'5',
          'format':'json'}

# map languages
lang_upper = {'German': 'de',
        'English': 'en',
        'French': 'fr',
        'Dutch': 'nl',
        'Japanese': 'ja',
        'Chinese': 'zh',
        'Spanish': 'es',
        'Arabic': 'ar',
        'Hindi': 'hi',
        'Portugese': 'pt',
        'Italian': 'it',
        'Russian': 'ru'
       }

lang_lower = {'german': 'de',
        'english': 'en',
        'french': 'fr',
        'dutch': 'nl',
        'japanese': 'ja',
        'chinese': 'zh',
        'spanish': 'es',
        'arabic': 'ar',
        'hindi': 'hi',
        'portugese': 'pt',
        'italian': 'it',
        'russian': 'ru'
       }

# map common synonyms to correct properties
rules = {'name': 'rdfs:label',
         'scientific': 'P225',
         'pregnant': 'P3063',
         'weigh': 'P2067',
         #'eat': 'P1034',
         'tall': 'P2048',
         'old': 'P4214',
         'make': 'P1672',
         'come': 'P183',
         'extinct': 'P582',
         'study': 'P2579',
         #'consist': 'P527',
         'component':'P527',
         'endemic':'P183',
         'edible': 'Q9323487',
         'fur': 'P462'
        }

2022-06-13 14:51:32.438223: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-13 14:51:32.438242: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [49]:
#Visualization 
# displacy.render(parse, jupyter=True, style="ent")

# displacy.render(parse, jupyter=True, style="dep")

In [2]:
# Helper function to iterate through the rules, no matter the type
def checkRules(lemma, rules, outputList):
    if lemma in rules:
        if type(rules.get(lemma)) is list:
            for item in rules.get(lemma):
                outputList.append(item)
        else:
            outputList.append(rules.get(lemma))

In [3]:
# Maria's code
 

# # If there is a 'too many requests error' its probably in these 2 functions
def getLabel(query): 
    url = 'https://query.wikidata.org/sparql'
    
    try:
        results = requests.get(url, params={'query': query, 'format': 'json'}).json()
    except ValueError:
        return None
    
    for item in results['results']['bindings']:
        for var in item:
            print('{}\t{}'.format(var,item[var]['value']))


def getResults(query, label_query):
    
    url = 'https://query.wikidata.org/sparql'
    
    try:
        results = requests.get(url, params={'query': query, 'format': 'json'}).json()
    except ValueError:
        return None
    
    for item in results['results']['bindings']:
        for var in item:
            print('{}\t{}'.format(var,item[var]['value']))
            if type(item[var]['value']) == str:
                getLabel(label_query)
        return item[var]['value']
    return 'null'

def checkCompound(parse, token):
    for i, word in enumerate(parse): 
        if word.dep_ == "compound" and str(parse[i+1]) == token.text:
            compound = word
            return compound
    
    return 0

def checkAmod(parse, token):
    for i, word in enumerate(parse):
        if word.dep_ == "amod" and str(parse[i+1]) == token.text:
            amod = word
            return amod
    
    return 0

def checkAcomp(parse, token):
    for i, word in enumerate(parse):
        if word.dep_ == "acomp" and str(parse[i+1]) == token.text:
            if word.text == "strong":
                acomp = "force quotient"
                return acomp
    return 0


def checkEntityAdj(parse, token_text):
    for i, word in enumerate(parse):
        if word.pos_ == "ADJ" and i != len(parse)-1:
            if parse[i+1] == token_text:
                entAdj = word
                return entAdj
    
    return 0
    



def getID(token):
    params = {'action':'wbsearchentities', 
           'language':'en',
           'format':'json'}
    print(token.text)
    compound = checkCompound(parse, token)
    if token.dep_ != "amod":
        amod = checkAmod(parse, token)
    else:
        amod = 0
    
    entAdj = checkEntityAdj(parse, token)

    if type(compound) != int :
        text = compound.text + ' ' + token.text
    elif type(amod) != int:
        text = amod.text + ' ' + token.text
    
    elif type(entAdj) != int:
        text = entAdj.text + ' ' + token.text
    else:
        text = token.text
        
    params['search'] = text
    try:
        json = requests.get(url,params).json()
    except ValueError:
        return None

    # If its not initialized then initialize ID here as 0 or something and then check if its an int 
    for result in json['search']:
        if result['label'] == text:
            ID = result['id']
            return ID
        elif text in result['label']:
            ID = result['id']
            return ID    
    return 

# # Similarly to the getID() function, this one returns the ID of the property
def getID_property(token):
    params = {'action':'wbsearchentities', 
              'type':'property',
              'language':'en',
              'format':'json'}
    print(token.text)
    compound = checkCompound(parse, token)
    acomp = checkAcomp(parse, token)
    
    if type(compound) != int :
        text = compound.text + ' ' + token.text
    elif type(acomp) != int :
        text = token.text + ' ' + acomp
    else:
        text = token.text

    params['search'] = text
    
    json = requests.get(url,params).json()

    # If its not initialized then initialize ID here as 0 or something and then check if its an int 
    for result in json['search']:
        
        if result['label'] == text:
            ID = result['id']
            return ID

        elif text in result['label']:
            ID = result['id']
            return ID

  
    return 

def getID_property_str(token_text): # The same as getID_property, but it takes string instead of token as an argument
    params = {'action':'wbsearchentities', 
              'type':'property',
              'language':'en',
              'format':'json'}
    

    text = token_text
    

    params['search'] = text
    
    json = requests.get(url,params).json()
    # If its not initialized then initialize ID here as 0 or something and then check if its an int 
    for result in json['search']:
        
        if result['label'] == text:
            ID = result['id']
            return ID
            break
        elif text in result['label']:
            ID = result['id']
            return ID
            break
  
    return 

def tryAgain(question):
    ID1_token = ''
    ID2_token = ''
    ID2_token_text = 0   
    # Parsing the question and printing values to check - 
    # since I solved the previous assignment with dependencies, 
    # the code worked for both "What" and "How" questions, thus 
    # I extended the program with "Where" and "When" questions

    # Where and When questions are pretty specific in terms of 
    # plants and animals thus I added some extra conditions

    if str(parse[0]) == "Where": # Finding the token of the property and the entity for Where questions
        ID2_token_text = "endemic to"
        for word in parse:
            if word.dep_ == "nsubj" and word.text != 'you':
                ID1_token = word  
            elif word.dep_ == "dobj":
                ID1_token = word

    if str(parse[0]) == "When": # Finding the token of the property and the entity for When questions

        for word in parse:
            if word.dep_ == "nsubj":
                ID1_token = word  
            if word.dep_ == "acomp":
                if word.text == 'extinct':
                    ID2_token_text = 'extinction date'
                elif word.text == "alive" or word.text == "living":
                    ID2_token_text = 'start time'

    elif str(parse[0]) == "How" or str(parse[0]) == "What":
        for word in parse : # iterate over the token objects 
            print(word.text, word.lemma_, word.pos_, word.dep_)

            if word.dep_ == "nsubj" :
                ID2_token = word    # Finding the token of the property for What and How questions

            if word.dep_ == "pobj":
                ID1_token =  word   # Finding the token of the entity for What and How questions


    for ent in parse.ents : 
        print(ent.text, ent.label_)
    
    if ID1_token and ID2_token:
        ID1 = getID(ID1_token)
        if type(ID2_token_text) != int:
            ID2 = getID_property_str(ID2_token_text)
        else:
            ID2 = getID_property(ID2_token)

        if ID1 is None or ID2 is None:
            return 
    else:
        return
    query = 'SELECT ?answer WHERE { wd:' + ID1 + ' wdt:' + ID2 + ' ?answer}'

    # If the answer is an URL, this query will serve to also print the label 
    label_query = 'SELECT ?answerLabel WHERE { wd:' + ID1 + ' wdt:' + ID2 + ' ?answer. ?answer rdfs:label ?answerLabel. FILTER(lang(?answerLabel)="en")}LIMIT 1'

    print(question)
    return getResults(query, label_query)

In [4]:
# Bianca's code:

def getAnswer(question):

    # arrays/lists to save all found IDs/URIs of entities and properties
    ents = []
    props = []

    langFilter = ''
    addition = ''
    rootFlag = False
    ofOrFromInSentence = False
    objInSentence = False
    yesNoQuestion = False
    countQuestion = False
    noSubject = True
    flip_answer = False 
    languageFound = ''

    # parse the question 
    parse = nlp(question) 

    # use entities to detect languages - what about e.g., "German Shepard"?
    for ent in parse.ents:
        if ent.label_ == 'NORP':
            languageFound = ent
            print('language found: ' + ent.lemma_)
            print(ent.lemma_)
            if ent.lemma_ in lang_lower.keys():
                langFilter = 'FILTER (lang(?answer) = "' + str(lang_lower.get(ent.lemma_)) + '")'
            elif ent.lemma_ in lang_upper.keys():
                langFilter = 'FILTER (lang(?answer) = "' + str(lang_upper.get(ent.lemma_)) + '")'
            print(langFilter)


    # parse once to check question structure - add question qualification (what, where, who etc.) here
    for word in parse:
        print(word.text, word.lemma_, word.pos_, word.dep_)
        if word.text == str(languageFound) and parse[word.i + 1].text.istitle():
            langFilter = ''
            languageFound = ''
        # if the question contains 'of' or 'from' and they are not the last word in the question
        if (word.text == 'of' or word.text == 'from') and word.text != parse[-2].text:
            ofOrFromInSentence = True
        if 'different from' in question:
            flip_answer = True   
        if word.dep_ == 'nsubj' or word.dep_ == 'nsubjpass':
            noSubject = False
        # if the question contains an object
        if word.dep_ == 'pobj' or word.dep_ == 'dobj' or word.dep_ == 'attr' or word.dep_ == 'acomp':
            objInSentence = True
        # if the question is a count question
        if word.text == 'many':
            countQuestion = True
    # parse once again to check question structure, this is needed due to the possibility of having an object at the last position
    for word in parse:
        if parse[0].pos_ == 'VERB' or parse[0].pos_ == 'AUX':
            rootFlag = False
            yesNoQuestion = True
            break
        # if the voice is passive, the property has to be an nsubj or nsubjpass
        if word.dep_ == 'auxpass':
            rootFlag = False
            break
        # if the verb is the ROOT or could be the case that the question contains modal verbs, e.g. 
        # 'Which colour can foxes be?', then the entity has to be an nsubj'
        if word.pos_ == 'VERB' or (word.pos_ == 'AUX' and word.text == parse[-2].text):
            if word.dep_ == 'ROOT':
                rootFlag = True
            else:
                if not ofOrFromInSentence:
                    rootFlag = True       

    # for 'endemic to' examples and other ones
    if not yesNoQuestion and not rootFlag and not objInSentence:
        rootFlag = True          

    # iterate through parsed words
    for word in parse:
        # check for specific rules, synonyms etc
        if not yesNoQuestion:
            checkRules(word.lemma_, rules, props)
        else:
            if not str(rules.get(word.lemma_)).startswith('P'):
                ents.append(rules.get(word.lemma_))

        # iterate through word positions
        if word.pos_:
            # if adjective or adverb, extend words for websearch
            if word.pos_ == 'ADJ' or word.pos_ == 'ADV':
                if word.dep_ == 'amod' or word.dep_ == 'compound' or word.dep_ == 'advmod':
                    addition = addition + ' ' + word.text

            # if noun, search for URI (entity), searches for whole words only at this point - needs adjustments for spans
            if word.pos_ == 'NOUN' or word.pos_ == 'PROPN' or word.pos_ == 'NUM' or word.pos_ == 'VERB' or word.pos_ == 'ADJ':
                if word.dep_ == 'compound':
                    # maybe not necessary
                    if word.pos_ == 'NOUN' or word.pos_ == 'PROPN':
                         addition = addition + ' ' + word.lemma_
                    else: 
                        addition = addition + ' ' + word.text

                if not rootFlag:
                    # if pobj, an entity has been found
                    if word.dep_ == 'pobj' or word.dep_ == 'dobj' or word.dep_ == 'attr' or word.dep_ == 'acomp':
                        # for 'endemic to'
                        if word.lemma_ in rules and addition:
                            entParams['search'] = addition
                        else:
                            entParams['search'] = addition + ' ' + word.lemma_
                        json = requests.get(url,entParams, headers=headers).json()
                        for result in json['search']:
                            if 'description' in result:
                                # save all IDs of interest for entities
                                ents.append(result['id'])
                        # if nothing could be found with the extension, try shortened version
                        addition = ''
                        if not ents:
                            entParams['search'] = addition + ' ' + word.lemma_
                            json = requests.get(url,entParams, headers=headers).json()
                            for result in json['search']:
                                if 'description' in result:
                                    # save all IDs of interest for entities
                                    ents.append(result['id'])
                                    
                    # if nsubj, a property has been found
                    elif (word.dep_ == 'nsubj' or word.dep_ == 'nsubjpass' or word.dep_ == 'pcomp' or word.dep_ == 'ROOT' or word.dep_ == 'nmod') and word.pos_ != 'VERB':
                        if yesNoQuestion:
                            entParams['search'] = addition + ' ' + word.lemma_
                            json = requests.get(url,entParams, headers=headers).json()
                        else:
                            propParams['search'] = addition + ' ' + word.lemma_
                            json = requests.get(url,propParams, headers=headers).json()
                        for result in json['search']:
                            if 'description' in result:
                                # save all IDs of interest for properties
                                props.append(result['id'])
                        # added for average litter size example and other ones
                        if addition.rfind(' ') != 0:
                            addition = addition.split(" ", 2)[-1]
                        else:
                            addition = ''
                        # if nothing could be found with the extension, try shortened version
                        if not props or (yesNoQuestion and not objInSentence):
                            if yesNoQuestion:
                                entParams['search'] = addition + ' ' + word.lemma_
                                json = requests.get(url,entParams, headers=headers).json()
                            else:
                                propParams['search'] = addition + ' ' + word.lemma_
                                json = requests.get(url,propParams, headers=headers).json()
                            for result in json['search']:
                                if 'description' in result:
                                    # save all IDs of interest for properties
                                    if yesNoQuestion:
                                        ents.append(result['id'])
                                    else:
                                        props.append(result['id'])
                 # new addition
                    elif yesNoQuestion and not objInSentence and addition:
                        entParams['search'] = addition
                        json = requests.get(url,entParams, headers=headers).json()
                        for result in json['search']:
                            if 'description' in result:
                                # save all IDs of interest for properties
                                props.append(result['id'])

                    elif yesNoQuestion and noSubject and addition:
                        entParams['search'] = addition
                        json = requests.get(url,entParams, headers=headers).json()
                        for result in json['search']:
                            if 'description' in result:
                                # save all IDs of interest for properties
                                props.append(result['id'])

                # a ROOT verb is there, the syntax changes
                else:
                    # if nsubj or nsubjpass, an entity has been found
                    if word.dep_ == 'nsubj' or word.dep_ == 'nsubjpass':
                        entParams['search'] = addition + ' ' + word.lemma_
                        json = requests.get(url,entParams, headers=headers).json()
                        for result in json['search']:
                            if 'description' in result:
                                # save all IDs of interest for entities
                                ents.append(result['id'])
                        # if nothing could be found with the extension, try shortened version
                        if not ents:
                            if addition.istitle(): # added for Old Tjikko tree and other examples 
                                entParams['search'] = addition
                            else:
                                addition = ''
                                entParams['search'] = addition + ' ' + word.lemma_
                            json = requests.get(url,entParams, headers=headers).json()
                            for result in json['search']:
                                if 'description' in result:
                                    # save all IDs of interest for entities
                                    ents.append(result['id'])

                    if word.dep_ == 'pobj' or word.dep_ == 'dobj' or word.dep_ == 'attr' or word.dep_ == 'pcomp':
                        propParams['search'] = addition + ' ' + word.lemma_
                        json = requests.get(url,propParams, headers=headers).json()
                        for result in json['search']:
                            if 'description' in result:
                                # save all IDs of interest for properties
                                props.append(result['id'])
                        # I do not remember exactly for what examples was needed
                        if addition.rfind(' ') != 0:
                            addition = addition.split(' ', 2)[-1]
                        else:
                            addition = ''
                        # if nothing could be found with the extension, try shortened version
                        if not props:
                            propParams['search'] = addition + ' ' + word.lemma_
                            json = requests.get(url,propParams, headers=headers).json()
                            for result in json['search']:
                                if 'description' in result:
                                    # save all IDs of interest for properties
                                    props.append(result['id'])
                                    
                if not yesNoQuestion and word.pos_ == 'VERB':
                    # check for potential properties associated with the word - usually uses rules, though
                    if word.dep_ == 'ROOT':
                        propParams['search'] = addition + ' ' + word.lemma_
                        json = requests.get(url,propParams, headers=headers).json()
                        for result in json['search']:
                            if 'description' in result:
                                # save all IDs of interest for properties
                                props.append(result['id'])
                        addition = ''
                        # if nothing could be found with the extension, try shortened version
                        if not props:
                            propParams['search'] = word.lemma_
                            json = requests.get(url,propParams, headers=headers).json()
                            for result in json['search']:
                                if 'description' in result:
                                    # save all IDs of interest for properties n 
                                    props.append(result['id'])
    
    filtered_ents = list(filter(None, ents))
    filtered_props = list(filter(None, props))
    # maybe not necessary
    if yesNoQuestion:
        
        filtered_ents, filtered_props = filtered_props, filtered_ents
        
    print('found entitites: ', filtered_ents)
    print('found properties ', filtered_props)

    # load query url
    queryUrl = 'https://query.wikidata.org/sparql'
    # arrays/lists to save all found labels and values of the answer
    answers_labels = []
    answer_values = []
    yes_no_answer = []
    
    # iterate through entities and construct query
    for entIdx in filtered_ents:
        # iterate through properties and build the query
        for propIdx in filtered_props:
            # print('searching for entity ' + str(entIdx) + ' and property ' + str(propIdx))
            if not yesNoQuestion and langFilter and not countQuestion:
                query = 'SELECT ?answerLabel WHERE { wd:' + entIdx + ' rdfs:label' + ' ?answer.' + langFilter + ' SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } }'
            # new addition
            elif yesNoQuestion:
                query = 'ASK { wd:' + entIdx + ' wdt:P171*|wdt:P31|wdt:P279*|wdt:P1672|wdt:P141|wdt:P2283|wdt:P1034|wdt:P366|wdt:P462' + ' wd:' + propIdx + ' SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }}'                
            elif countQuestion:
                # this also covers disguised what questions
                query = getCountQuery(parse, entIdx, propIdx, langFilter)
            else:
                query = 'SELECT ?answerLabel WHERE { wd:' + entIdx + ' wdt:' + propIdx + ' ?answer. SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } }'
            
            # for sending too many requests
            try:
                data = requests.get(queryUrl, params={'query': query, 'format': 'json'}, headers=headers).json()
            except ValueError:
                continue

            if yesNoQuestion:
                yes_no_answer.append(data['boolean'])
            else:
                for item in data['results']['bindings']:
                    for var in item:
                        print('{}\t{}'.format(var, item[var]['value']))
                        answers_labels.append(var)
                        answer_values.append(item[var]['value'])
            # exit the function if an answer has been found as it is usually correct        
            if len(answers_labels) != 0 and len(answer_values) != 0:
                return ", ".join(answer_values)
            elif yesNoQuestion:
                if True in yes_no_answer:
                    if not flip_answer:
                        print("yes")
                        return "yes"
                    else:
                        print("no")
                        return "no"
                    return
    if yesNoQuestion:
        if not flip_answer:
            print("no")
            return "no"
        else:
            print("yes")
            return "yes"
        return
    elif len(answers_labels) == 0 and len(answer_values) == 0:
        return tryAgain(question)
        #print('No answer could be found')
        #return('No answer could be found')
        
# If it is run with input json file, this needs to be commented out
#question = input('Please ask your question\n')
#parse = nlp(question)
#getAnswer(question)

In [5]:
# Niclas' code
# the dependencies shown here differ from the ones in my juypter notebook - I need to adjust this. Shouldn't take long if the property is always 'attr'
def getCountQuery(parseQ, entIdx, propIdx, langFilter):
    ents = []
    props = []
    
    for word in parseQ:
        # check for specific rules, synonyms etc
        checkRules(word.lemma_, rules, props
                   )
        # sort the nsubjs to their respective POS/dependencies
        if word.dep_ == 'nsubj':
            # if it is the first (and only) nsubj, it should be the entity
            if ents:
                ents.append(word.lemma_)
            # if there is a second nsubj, the first nsubj becomes the entity
            # and the entity is replaced by the second nsubj 
            else:
                props.append(ents)
                ents.clear()
                ents.append(word.lemma_)

        # if adjective or adverb, or compund noun extend words for entity websearch
        if word.pos_ == 'ADJ' or word.pos_ == 'ADV' or word.pos_ == 'NOUN' or word.pos_ == 'PROPN':
            if word.dep_ == 'amod' or word.dep_ == 'compound' or word.dep_ == 'advmod':
                addition = addition + ' ' + word.text
                
        # if only one nsubj, the dobj is the property
        if word.dep_ == 'dobj':
            # this needs to find the synonyms for properties, 
            # such as pup(pie)s/eggs as synonyms for litter size
            ## if addition:
                ## props.append(classifier(addition))
            ## else:
                ## props.append(classifier(word.lemma_)) 
            # naive approach for now
            props.append(word.lemma_)
    
    # construct count query with corresponding entity and property indeces
    if langFilter:
        # try with normal what query first (disguised question)
        query = 'SELECT ?answer WHERE { wd:' + entIdx + ' wdt:' + propIdx + ' ?answer.' + langFilter + ' SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } }'
        # If the answer is an URL, this query will serve to also print the label 
        label_query = 'SELECT ?answerLabel WHERE { wd:' + entIdx + ' wdt:' + propIdx + ' ?answer. ?answer rdfs:label ?answerLabel. FILTER(lang(?answerLabel)="en")}LIMIT 1'
        # if this works, return the answer, otherwise do an actual counting operation
        if getResults(query, label_query):
            return getResults(query, label_query)
        else: 
            return 'SELECT (COUNT(?answer) as ?sum_of_answer) WHERE { wd:' + entIdx + ' wdt:' + propIdx + ' ?answer.' + langFilter + ' SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } }'

    else:
        # try with normal what query first (disguised question)
        query = 'SELECT ?answer WHERE { wd:' + entIdx + ' wdt:' + propIdx + ' ?answer. SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } }'
        label_query = 'SELECT ?answerLabel WHERE { wd:' + entIdx + ' wdt:' + propIdx + ' ?answer. ?answer rdfs:label ?answerLabel. FILTER(lang(?answerLabel)="en")}LIMIT 1'
        if getResults(query, label_query):
            return getResults(query, label_query)
        else: 
            return 'SELECT (COUNT(?answer) as ?sum_of_answer) WHERE { wd:' + entIdx + ' wdt:' + propIdx + ' ?answer. SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } }'
    

In [6]:
# store and print the input as question string
# question = input('Please ask your question\n')
# parse = nlp(question)

#This function is meant to run the code with a csv file as input, run only if thats the case
import json

input = open('test.json')
print("input: ", type(input))
input_data = input.read()
print("input_data: ", type(input_data))
data = json.loads(input_data)
output_data = []

for i in data:
    print("i: ", i)
    id = i['id']
    print("id: ", type(id))
    question = i['question']
    print("question: ", type(question))
    parse = nlp(question)

    dictionary ={
    "id" : id, 
    "question" : question, 
    "answer" : getAnswer(question),
    "correct" : ' '
  }
    
    output_data.append(dictionary)
  #json_object = json.dump(dictionary, indent = 4)

with open('test_output.json', "w", encoding='utf-8') as output:
    #output.write(json_object)
    json.dump(output_data, output, ensure_ascii=False, indent=4)


input:  <class '_io.TextIOWrapper'>
input_data:  <class 'str'>
i:  {'id': 1, 'question': 'What is the colour of milk?', 'answer': 'white', 'correct': 1}
id:  <class 'int'>
question:  <class 'str'>
What what PRON attr
is be AUX ROOT
the the DET det
colour colour NOUN nsubj
of of ADP prep
milk milk NOUN pobj
? ? PUNCT punct
found entitites:  ['Q8495', 'Q10988133']
found properties  ['P462', 'P1227', 'P1884', 'P465', 'P6338']
answerLabel	white
i:  {'id': 2, 'question': 'What are the parts of a tree?', 'answer': 'trunk, branch, leaf, root', 'correct': 0.5}
id:  <class 'int'>
question:  <class 'str'>
What what PRON attr
are be AUX ROOT
the the DET det
parts part NOUN nsubj
of of ADP prep
a a DET det
tree tree NOUN pobj
? ? PUNCT punct
found entitites:  ['Q10884', 'Q223655']
found properties  ['P1107', 'P1433', 'P59', 'P361', 'P179']
answerLabel	forest
answerLabel	phanerophyte
i:  {'id': 3, 'question': 'What is the size of an elephant?', 'answer': None, 'correct': 0}
id:  <class 'int'>
quest

found entitites:  ['Q9147', 'Q3374730']
found properties  ['P154', 'P5187', 'P676', 'P8814', 'P5191']
answerLabel	manchot
i:  {'id': 21, 'question': 'What is the average top speed of a cheetah?', 'answer': 'white', 'correct': 1}
id:  <class 'int'>
question:  <class 'str'>
What what PRON attr
is be AUX ROOT
the the DET det
average average ADJ amod
top top ADJ amod
speed speed NOUN nsubj
of of ADP prep
a a DET det
cheetah cheetah NOUN pobj
? ? PUNCT punct
found entitites:  ['Q23907', 'Q28647560']
found properties  []
What what PRON attr
is be AUX ROOT
the the DET det
average average ADJ amod
top top ADJ amod
speed speed NOUN nsubj
of of ADP prep
a a DET det
cheetah cheetah NOUN pobj
? ? PUNCT punct
cheetah
speed
What is the average top speed of a cheetah?
answer	120
i:  {'id': 22, 'question': 'What color is a daffodil?', 'answer': 'white', 'correct': 1}
id:  <class 'int'>
question:  <class 'str'>
What what DET det
color color NOUN nsubj
is be AUX ROOT
a a DET det
daffodil daffodil NOUN a

## Questions that work:

* What do cows eat?
* What is the top speed of a cheetah?
* What is the study of fish called?
* What does a plant consist of?
* What is the life expectancy of a sloth? -- there are two references, so two answers

* Which products are made from a sheep?

* How old can pandas become?
* How old can a European hedgehog get?
* How old can the Great Basin Bristlecone Pine get?

* Where do koalas come from?

* When did Mammoths go extinct? -- The date format is quite hard to read, needs polishing/units

* What is the german name for chicken?

* What is the dutch name for a bull?

* Where is a kangaroo endemic to?

* What is the heart rate of a giraffe in bpm?

* What is the French word for penguin?

Questions that do not work (yet):
1. Who was the polar bear born in captivity at the Berlin Zoological Garden in 2006? - Answer: Knut
The QUery should look something like this: 
'''SELECT ?nameLabel WHERE {
  ?name wdt:P10241 wd:Q33609 .
  ?name wdt:P19 wd:Q154828 .
  ?name wdt:P569 ?birthdate .
  FILTER (YEAR(?birthdate) <= 2007)
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
  }
}'''
2. Where are kangaroos endemic to? - does not work because the lemma of kangaroos is kangaroos since it is interpreted as PROPN

4. Which continent is the wombat endemic to? - does not work but works for Which continent is the common wombat endemic to?
5. How long is the gestation period of the European Rabbit? - works locally

* More questions :

1. What is the average top speed of a cheetah?
2. What is the highest observed lifespan in alpaca?
3. What is the diel cycle of a polar bear?
4. What are the parts of a tree? - does not give the correct answer because the property found is 'part' which is different from the 'parts' property
5. What is the life expectancy of a house cat? - works but 'What is the life expectancy of a cat?' does not work
6. What is the height of an elephant? and What is the average height of an elephant?
7. What is the gestation period of an Asian elephant? - works but What is the average gestation period of an Asian elephant? does not work
8. What is the habitat of an ivy?
9. What is the taxon rank of birch?
10. What is the main food source of an eagle?
11. What is the parent taxon of a lion?
12. What color is a daffodil?
13. What is the conservation status of the Atlantic salmon?
14. What is the average litter size of a jaguar?
15. Which color is a fox?
16. Which color is an elephant?
17. Which animals do not have a spine? - spine is not property on wikidata page
18. In which habitats do giraffes live? - works if it is habitat otherwise the dependency is wrong
19. In which habitat do tulips grow best?
20. In which country the International Plant Names Index is based? - works but In which country is the International Plant Names Index based? does not work because no subject is found 
21. In which country is the Old Tjikko tree located?
22. Which colour can foxes be? and Which colours can foxes be? 
23. Which animals are in the big 5? - finds the correct entity, but not the correct property
24. Which awards did Rosmarinus officinalis receive?
25. Which awards did Salvia rosmarinus receive? - why I added verb as a nsubj
26. Which awards did the rosemary receive? - why I added adj as a nsubj, without the is not working at all
27. Which type of fruit do rubber trees produce? - finds first an incorrect property
28. 
29. Which country is platypus endemic to?
30. 
31. How long is the gestation period of a cow?
32. Where is the Varanus komodoensis (komodo dragon) endemic to?
33. Where is the cocoa plant endemic to?
34. Which colours can the fruit of the apple tree have? - does not work but works for Which colours can the apple have?

New addition:

35. Are jaguars mammals?
36. Are bananas berries?
37. Is a lion a species of big cat?
38. Are paeonias different from pions?
39. Are elephants mammals?
40. Is cannabis different from marihuana?
41. Are bamboos edible plants? - does not work because the lemma of bamboos is bamboos since it is interpreted as ADJ but works for Are bamboo edible plants? and Is bamboo an edible plant?
42. Are sheep herbivores?
43. Do dolphins have a sense of self-awareness?
44. Do tigers eat fruit?
45. Is bamboo used inmaking clothing?
46. Are roses edible?
47. Can foxes have black fur?
48. Are fungi plants?


## Known bugs
Resolved? Sometimes the system sends too many requests - I'm already in conversation about this with Joost. 

I changed how the system gives an answer, and now I think there is only one question for which the system sends too many requests: 'In which habitats do giraffes live?'.

For some of the disguised questions, I need the classifier to find the corresponding synonyms

/////////////////////////////////////////////////////////////////////////////

FOR THE INPUT/OUTPUT:

So it basically messes up with writing down the result in the json file when the result is url. There are two ways i can think of for fixing it:

1. Either take only the label from the get label function as a return statement and record that. 

2. Or try and figure out how to record an url in a json file

Also, if there is time, check out if the questions that don't work in the output file but work if you input them manually
